In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
import os
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
AUTOTUNE = tf.data.experimental.AUTOTUNE

Using TensorFlow backend.


In [77]:
ls1 = os.listdir(r"C:\Users\Arnab Sinha\Documents\GitHub\IEEE-Project-Agribot\data\color")

In [78]:
len(ls1)

38

In [79]:
import os
ls1=os.listdir(r"C:\Users\Arnab Sinha\Documents\GitHub\IEEE-Project-Agribot\data\color")
dic1={}
import scipy.misc as sm
import numpy as np
count=0
for idx,i in enumerate(ls1):
    dic1[i]=idx
    ls2=os.listdir(r'C:\Users\Arnab Sinha\Documents\GitHub\IEEE-Project-Agribot\data\color/' + i)
    for j in ls2:
        #im1=np.asarray(sm.imread('color/'+i+'/'+j))
        #temp=np.zeros((len(im1),len(im1[0]),len(im1[0][0])   ))
        count=count+1

In [80]:
count

54305

In [81]:
dic1={}
import scipy.misc as sm
import numpy as np
X=[]
Y=np.zeros((count,1))
vap=0

for idx,i in enumerate(ls1):
    dic1[i]=idx
    ls2=os.listdir(r"C:\Users\Arnab Sinha\Documents\GitHub\IEEE-Project-Agribot\data\color/" + i)
    for j in ls2:
        s = r"C:\Users\Arnab Sinha\Documents\GitHub\IEEE-Project-Agribot\data\color/" + i + '\\' + j
        X.append(s)
        Y[vap,0]=idx
        vap=vap+1    

In [7]:
def preprocess_image(image):
    image = tf.image.decode_jpeg(tf.io.read_file(image))
    image = tf.image.resize(image, [300, 300])
    image /= 255.0  # normalize to [0,1] range
    return image

In [8]:
Y.shape

(54305, 1)

In [45]:
batch_size = 32
num_classes = len(dic1)
epochs = 1

# input image dimensions
img_rows, img_cols = 300, 300

In [10]:
ind=np.random.permutation(len(X))
len_ind=ind.shape[0]
len_ind

54305

In [55]:
train_ind= ind[0: int(0.05*len_ind)]
val_ind= ind[ int(0.05*len_ind) : int(0.1*len_ind)]
test_ind= ind[ int(0.1*len_ind) : int(0.125*len_ind)]

In [56]:
len(train_ind)

2715

In [57]:
X2 = []
for item in ind:
    X2.append(X[item])
    
Y2 = []
for item in ind:
    Y2.append(Y[item])

In [63]:
X = X2
Y = [int(a) for a in Y2]

In [64]:
X_train=X[0:int(0.05*len_ind)]
X_val=X[int(0.05*len_ind):int(0.1*len_ind)]
X_test=X[int(0.1*len_ind) :int(0.125*len_ind)]

In [66]:
X_train[1000]

'C:\\Users\\Arnab Sinha\\Documents\\GitHub\\PlantVillage-Dataset\\raw\\color/Orange___Haunglongbing_(Citrus_greening)\\8a22a0a3-1858-432b-b2e8-e54a22d4efea___CREC_HLB 4173.JPG'

In [67]:
Y_train=Y[0:int(0.05*len_ind)]
Y_val=Y[int(0.05*len_ind):int(0.1*len_ind)]
Y_test=Y[int(0.1*len_ind) :int(0.125*len_ind)]

In [68]:
# convert class vectors to binary class matrices
Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_test = keras.utils.to_categorical(Y_test, num_classes)
Y_val = keras.utils.to_categorical(Y_val, num_classes)

In [20]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(300,300,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        3

In [22]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [39]:
BATCH_SIZE = 32

def make_dataset(X,Y,length):
       
    path_ds = tf.data.Dataset.from_tensor_slices(X)

    image_ds = path_ds.map(preprocess_image, num_parallel_calls=AUTOTUNE)

    label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(Y, tf.int64))

    image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))

    print(image_label_ds)

    # ds = tf.data.Dataset.from_tensor_slices((X,Y))

    # Setting a shuffle buffer size as large as the dataset ensures that the data is
    # completely shuffled.
    ds = image_label_ds.shuffle(buffer_size=length)
    ds = ds.repeat()
    ds = ds.batch(BATCH_SIZE)
    # `prefetch` lets the dataset fetch batches in the background while the model is training.
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

In [69]:
ds_train = make_dataset(X_train,Y_train,len(train_ind))
ds_val = make_dataset(X_val,Y_val,len(val_ind))
ds_test = make_dataset(X_test,Y_test,len(test_ind))

<ZipDataset shapes: ((300, 300, None), (38,)), types: (tf.float32, tf.int64)>
<ZipDataset shapes: ((300, 300, None), (38,)), types: (tf.float32, tf.int64)>
<ZipDataset shapes: ((300, 300, None), (38,)), types: (tf.float32, tf.int64)>


In [31]:
# ds = image_label_ds.apply(
#   tf.data.experimental.shuffle_and_repeat(buffer_size=len(train_ind)))
# ds = ds.batch(BATCH_SIZE)
# ds = ds.prefetch(buffer_size=AUTOTUNE)

In [48]:
model.fit(ds_train,
          epochs=epochs,
          verbose=1,
          validation_data=ds_val,
          steps_per_epoch = 170,
          validation_steps = 40,
          validation_freq = 1)

170/170 [==============================] - 1631s 10s/step - loss: 3.6295 - accuracy: 0.0395 - val_loss: 3.6239 - val_accuracy: 0.1031


In [71]:
score = model.evaluate(ds_test, verbose=1,steps = 20)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

20/20 [==============================] - 73s 4s/step - loss: 3.6263 - accuracy: 0.0844
Test loss: 3.626318621635437
Test accuracy: 0.084375


In [74]:
model.save("my_model.h5") #model saved

In [75]:
new_model = tf.keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        3